In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
!pip install -q albumentations opencv-python-headless pillow tqdm pyyaml

In [15]:
import cv2
import numpy as np
import albumentations as A
from pathlib import Path
from tqdm import tqdm
import shutil
import yaml
import json
from ultralytics import YOLO

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [14]:
!pip install ultralytics


In [16]:
def get_augmentation_pipeline():
    """Augmentation pipeline for drone landing conditions"""
    return A.Compose([
        # Lighting variations
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1),
            A.RandomGamma(gamma_limit=(50, 150), p=1),
            A.CLAHE(p=1),
        ], p=0.7),
        # Weather effects
        A.OneOf([
            A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=1),
            A.RandomRain(slant_lower=-10, slant_upper=10, p=1),
            A.RandomSunFlare(src_radius=100, p=1),
        ], p=0.3),
        # Motion blur
        A.OneOf([
            A.MotionBlur(blur_limit=15, p=1),
            A.GaussianBlur(blur_limit=7, p=1),
            A.Defocus(radius=(3, 7), alias_blur=(0.1, 0.3), p=1),
        ], p=0.4),
        # Noise
        A.OneOf([
            A.GaussNoise(var_limit=(10, 50), p=1),
            A.ISONoise(p=1),
            A.CoarseDropout(max_holes=5, max_height=50, max_width=50, p=1),
        ], p=0.3),
        # Geometric
        A.OneOf([
            A.Rotate(limit=15, p=1),
            A.Perspective(scale=(0.05, 0.1), p=1),
        ], p=0.3),
        # Color shifts
        A.OneOf([
            A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=1),
        ], p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def simulate_night_vision(image):
    """Create night vision effect"""
    dark = cv2.convertScaleAbs(image, alpha=0.3, beta=-50)
    gray = cv2.cvtColor(dark, cv2.COLOR_BGR2GRAY)
    night = cv2.applyColorMap(gray, cv2.COLORMAP_BONE)
    noise = np.random.normal(0, 15, night.shape).astype(np.uint8)
    return cv2.add(night, noise)

def add_lens_occlusion(image, severity='medium'):
    """Simulate dust on lens"""
    overlay = image.copy()
    h, w = image.shape[:2]
    num_spots = {'light': 2, 'medium': 4, 'heavy': 7}[severity]
    
    for _ in range(num_spots):
        x = np.random.randint(0, w)
        y = np.random.randint(0, h)
        radius = np.random.randint(30, 80)
        cv2.circle(overlay, (x, y), radius, (180, 160, 140), -1)
    
    return cv2.addWeighted(image, 0.6, overlay, 0.4, 0)

print("✅ Augmentation functions defined!")

✅ Augmentation functions defined!


In [17]:
def augment_dataset(input_dir, output_dir, num_augmentations=3):
    """
    Augment dataset
    Args:
        input_dir: Path to original data (should contain 'images' and 'labels' folders)
        output_dir: Where to save augmented data
        num_augmentations: How many augmented versions per image
    """
    transform = get_augmentation_pipeline()
    
    images_in = Path(input_dir) / 'images'
    labels_in = Path(input_dir) / 'labels'
    images_out = Path(output_dir) / 'images'
    labels_out = Path(output_dir) / 'labels'
    
    images_out.mkdir(parents=True, exist_ok=True)
    labels_out.mkdir(parents=True, exist_ok=True)
    
    image_files = list(images_in.glob('*.*'))
    print(f"Found {len(image_files)} images to augment...")
    
    for img_path in tqdm(image_files, desc="Augmenting"):
        # Load image
        image = cv2.imread(str(img_path))
        if image is None:
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load labels
        label_path = labels_in / f"{img_path.stem}.txt"
        bboxes = []
        class_labels = []
        
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_labels.append(int(parts[0]))
                    bboxes.append(list(map(float, parts[1:5])))
        
        # Copy original
        shutil.copy(img_path, images_out / img_path.name)
        if label_path.exists():
            shutil.copy(label_path, labels_out / label_path.name)
        
        # Create augmentations
        for aug_idx in range(num_augmentations):
            try:
                if aug_idx < num_augmentations - 1:
                    # Standard augmentation
                    augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                    aug_image = augmented['image']
                    aug_bboxes = augmented['bboxes']
                    aug_labels = augmented['class_labels']
                elif aug_idx == num_augmentations - 1 and np.random.random() > 0.5:
                    # Night vision
                    aug_image = simulate_night_vision(image)
                    aug_bboxes = bboxes
                    aug_labels = class_labels
                else:
                    # Lens occlusion
                    aug_image = add_lens_occlusion(image)
                    aug_bboxes = bboxes
                    aug_labels = class_labels
                
                # Save image
                aug_name = f"{img_path.stem}_aug{aug_idx}{img_path.suffix}"
                aug_image_bgr = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)
                cv2.imwrite(str(images_out / aug_name), aug_image_bgr)
                
                # Save labels
                label_out = labels_out / f"{img_path.stem}_aug{aug_idx}.txt"
                with open(label_out, 'w') as f:
                    for cls_id, bbox in zip(aug_labels, aug_bboxes):
                        f.write(f"{cls_id} {' '.join(map(str, bbox))}\n")
            except Exception as e:
                print(f"Error on {img_path.name}: {e}")

print("✅ Augmentation pipeline ready!")

✅ Augmentation pipeline ready!


In [18]:
INPUT_TRAIN = '/kaggle/input/dataset-hacktu/dataset/train'
INPUT_VALID = '/kaggle/input/dataset-hacktu/dataset/valid'


In [19]:
OUTPUT_TRAIN = '/kaggle/working/augmented/train'
OUTPUT_VALID = '/kaggle/working/augmented/valid'

In [20]:
print("Checking paths...")
print(f"Train exists: {Path(INPUT_TRAIN).exists()}")
print(f"Valid exists: {Path(INPUT_VALID).exists()}")

Checking paths...
Train exists: True
Valid exists: True


In [21]:
print("\nAvailable datasets:")
!ls /kaggle/input/dataset-hacktu


Available datasets:
dataset


In [ ]:
print("="*60)
print("STARTING DATA AUGMENTATION")
print("="*60)
print("\nThis will take 10-30 minutes...\n")

# Augment training data
print("1. Augmenting training set (3 versions per image)...")
augment_dataset(INPUT_TRAIN, OUTPUT_TRAIN, num_augmentations=3)

# Augment validation data
print("\n2. Augmenting validation set (2 versions per image)...")
augment_dataset(INPUT_VALID, OUTPUT_VALID, num_augmentations=2)

# Show results
original_train = len(list(Path(f"{INPUT_TRAIN}/images").glob('*.*')))
augmented_train = len(list(Path(f"{OUTPUT_TRAIN}/images").glob('*.*')))
original_valid = len(list(Path(f"{INPUT_VALID}/images").glob('*.*')))
augmented_valid = len(list(Path(f"{OUTPUT_VALID}/images").glob('*.*')))

print("\n" + "="*60)
print("AUGMENTATION COMPLETE!")
print("="*60)
print(f"Training:   {original_train} → {augmented_train} ({augmented_train/original_train:.1f}x)")
print(f"Validation: {original_valid} → {augmented_valid} ({augmented_valid/original_valid:.1f}x)")


/tmp/ipykernel_55/2157672548.py:12: UserWarning: Argument(s) 'fog_coef_lower, fog_coef_upper' are not valid for transform RandomFog
  A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=1),
/tmp/ipykernel_55/2157672548.py:13: UserWarning: Argument(s) 'slant_lower, slant_upper' are not valid for transform RandomRain
  A.RandomRain(slant_lower=-10, slant_upper=10, p=1),
/tmp/ipykernel_55/2157672548.py:24: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10, 50), p=1),
/tmp/ipykernel_55/2157672548.py:26: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=5, max_height=50, max_width=50, p=1),


STARTING DATA AUGMENTATION

This will take 10-30 minutes...

1. Augmenting training set (3 versions per image)...
Found 2880 images to augment...


Augmenting:   1%|          | 16/2880 [00:08<23:19,  2.05it/s]

Error on frame_008792.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].
Error on frame_008792.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].


Augmenting:   2%|▏         | 69/2880 [00:32<21:48,  2.15it/s]

...

Augmenting:  77%|███████▋  | 2226/2880 [20:06<04:56,  2.21it/s]

Error on frame_008628.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].
Error on frame_008628.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].


Augmenting:  83%|████████▎ | 2386/2880 [21:28<05:03,  1.63it/s]

Error on frame_008616.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].
Error on frame_008616.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].


Augmenting:  84%|████████▎ | 2405/2880 [21:35<02:05,  3.78it/s]

In [26]:
!ls /kaggle/working/augmented/train/labels | wc -l


11178


In [27]:
ls /kaggle/working


augmented/


In [31]:
!mv /kaggle/working/augmented_valid.zip /kaggle/working/augmented_valid_v2.zip


In [33]:
from IPython.display import FileLink
FileLink('/kaggle/working/augmented_valid_v2.zip')


/kaggle/working/augmented_valid_v2.zip

In [22]:
def is_valid_yolo_bbox(b):
    _, x, y, w, h = b
    return 0 < w <= 1 and 0 < h <= 1 and 0 <= x <= 1 and 0 <= y <= 1

# remove invalid labels



In [28]:
data_config = {
    'path': '/kaggle/working/augmented',
    'train': 'train/images',
    'val': 'valid/images',
    'nc': 4,
    'names': ['Vehicle', 'UAP', 'UAI', 'Person']
}

yaml_path = '/kaggle/working/augmented/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)



In [34]:
!rm /kaggle/working/augmented_valid_v2.zip


In [35]:
!zip -r -s 200m /kaggle/working/augmented_valid_small.zip /kaggle/working/augmented/valid


  adding: kaggle/working/augmented/valid/ (stored 0%)
  adding: kaggle/working/augmented/valid/images/ (stored 0%)
  adding: kaggle/working/augmented/valid/images/frame_004504.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_006208_aug0.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_006316_aug0.jpg (deflated 1%)
  adding: kaggle/working/augmented/valid/images/frame_001964_aug1.jpg (deflated 1%)
  adding: kaggle/working/augmented/valid/images/frame_011008_aug1.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_003284_aug1.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_008364.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_014196_aug0.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_006664_aug0.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_002580.jpg (deflated 0%)
  adding: kaggle/working/augmented/valid/images/frame_011332

In [ ]:
!cd /kaggle/working
!zip -r augmented_valid.zip augmented/valid


  adding: augmented/valid/ (stored 0%)
  adding: augmented/valid/images/ (stored 0%)
  adding: augmented/valid/images/frame_004504.jpg (deflated 0%)
  adding: augmented/valid/images/frame_006208_aug0.jpg (deflated 0%)
  adding: augmented/valid/images/frame_006316_aug0.jpg (deflated 1%)
  adding: augmented/valid/images/frame_001964_aug1.jpg (deflated 1%)
  adding: augmented/valid/images/frame_011008_aug1.jpg (deflated 0%)
  adding: augmented/valid/images/frame_003284_aug1.jpg (deflated 0%)
  adding: augmented/valid/images/frame_008364.jpg (deflated 0%)
  adding: augmented/valid/images/frame_014196_aug0.jpg (deflated 0%)
  adding: augmented/valid/images/frame_006664_aug0.jpg (deflated 0%)
  adding: augmented/valid/images/frame_002580.jpg (deflated 0%)
  adding: augmented/valid/images/frame_011332.jpg (deflated 0%)
  adding: augmented/valid/images/frame_007704_aug0.jpg (deflated 0%)
  adding: augmented/valid/images/frame_006332_aug0.jpg (deflated 0%)
  adding: augmented/valid/images/frame

In [ ]:
!ls /kaggle/input/uav-detector/other/default/1

In [ ]:
PHASE1_MODEL_PATH = '/kaggle/input/uav-detector/other/default/1/best.pt'

print("="*60)
print("TRAINING ON AUGMENTED DATA")
print("="*60)

# Load Phase 1 model
model = YOLO(PHASE1_MODEL_PATH)

# Train
print("\nStarting training (2-4 hours on GPU)...\n")
results = model.train(
    data='/kaggle/working/augmented/data.yaml',
    epochs=75,
    imgsz=640,
    batch=16,
    name='augmented_v1',
    patience=15,
    save=True,
    plots=True,
    device=0,
    workers=4,
    exist_ok=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    mosaic=1.0
)

# Validate
metrics = model.val()

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"mAP@0.5:     {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"Precision:    {metrics.box.p:.4f}")
print(f"Recall:       {metrics.box.r:.4f}")

# Save metrics
metrics_dict = {
    'map50': float(metrics.box.map50),
    'map': float(metrics.box.map),
    'precision': float(metrics.box.p),
    'recall': float(metrics.box.r)
}

with open('/kaggle/working/augmented_metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=2)

print("\n✅ Model saved to:")
print("   /kaggle/working/runs/detect/augmented_v1/weights/best.pt")

In [ ]:
!ls /kaggle/working/runs/detect/val

In [ ]:
from ultralytics import YOLO

model = YOLO("/kaggle/working/runs/detect/augmented_v1/weights/best.pt")
metrics = model.val(data='/kaggle/working/augmented/data.yaml')

print(f"mAP@0.5:      {metrics.box.map50.mean():.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map.mean():.4f}")
print(f"Precision:    {metrics.box.p.mean():.4f}")
print(f"Recall:       {metrics.box.r.mean():.4f}")


In [ ]:
metrics_dict = {
    'map50': float(metrics.box.map50.mean()),
    'map': float(metrics.box.map.mean()),
    'precision': float(metrics.box.p.mean()),
    'recall': float(metrics.box.r.mean())
}

with open('/kaggle/working/augmented_metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=2)

print("✅ Metrics JSON saved")


In [ ]:
!ls /kaggle/working/runs/detect/augmented_v1/weights


In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Load trained model
test_model = YOLO('/kaggle/working/runs/detect/augmented_v1/weights/best.pt')

# Get sample validation images (augmented dataset)
val_images = list(Path('/kaggle/working/augmented/valid/images').glob('*.jpg'))[:5]

print("Testing model on sample images...\n")

for img_path in val_images:
    results = test_model(str(img_path))
    print(f"{img_path.name}:")
    
    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            cls = int(box.cls.item())
            conf = float(box.conf.item())
            print(f"  Class {cls}, Confidence: {conf:.3f}")
    else:
        print("  No detections")

print("\n✅ Phase 2 Complete!")


In [ ]:
!ls /kaggle/working/runs/detect


In [ ]:
!zip -r /kaggle/working/yolo_runs_detect.zip /kaggle/working/runs/detect


In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Load model
model = YOLO("/kaggle/working/runs/detect/augmented_v1/weights/best.pt")

# Class names (must match training order)
CLASS_NAMES = ['Vehicle', 'UAP', 'UAI', 'Person']

# Select sample images
image_paths = list(Path("/kaggle/working/augmented/valid/images").glob("*.jpg"))[:5]

for img_path in image_paths:
    results = model(str(img_path), conf=0.25)
    print(f"\n🖼 Image: {img_path.name}")

    if len(results[0].boxes) == 0:
        print("  ❌ No detections")
        continue

    for box in results[0].boxes:
        cls_id = int(box.cls.item())
        conf = float(box.conf.item())
        print(f"  ✅ {CLASS_NAMES[cls_id]} — Confidence: {conf:.3f}")
